In [6]:
import requests

In [3]:
#!python -m http.server

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
^C

Keyboard interrupt received, exiting.


In [56]:
client_id = '1fvo2c155aknlf8i37h7p6qeri'
client_secret = 'oqm61c2725jpfbo2i4s18t12c0'
base_api_url = 'https://api.meetup.com/gql'
redirect_uri = 'https://event-collector-api.herokuapp.com/'

auth_url = f'https://secure.meetup.com/oauth2/authorize'
access_url = f'https://secure.meetup.com/oauth2/access'

In [44]:
grant_type = 'anonymous_code'
headers = {'Accept': 'application/json'}
auth_params = {'client_id': client_id,
               'response_type': grant_type,
               'redirect_uri': redirect_uri}

In [45]:
auth_response = requests.get(auth_url, params=auth_params, headers=headers)

In [46]:
auth_response.json()

{'code': '801d2ccbeae6b9998bce98758a81d187'}

In [47]:
auth_token = auth_response.json()["code"]

In [48]:
access_params = {'client_id': client_id,
                         'client_secret': client_secret,
                         'grant_type': grant_type,
                         'redirect_uri': redirect_uri,
                         'code': auth_token}

In [49]:
access_response = requests.post(access_url, params=access_params, headers=headers)

In [50]:
access_token = access_response.json()
auth_string = 'Bearer %s' % access_token
oauth_headers = {'Accept': 'application/json', 'Authorization': auth_string}

In [54]:
oauth_headers

{'Accept': 'application/json',
 'Authorization': "Bearer {'access_token': 'a406572e929965c5707dcaa9486e878a', 'refresh_token': 'fd14e5d6ea168ace11bad3543179372b', 'member': {'country': 'at', 'city': 'Vienna', 'name': 'prereg_member_5768072516400', 'lon': 16.372696, 'id': 362296471, 'state': '09', 'lat': 48.1933}, 'token_type': 'bearer', 'expires_in': 3600}"}

In [107]:
activity = 'fitness'

In [108]:
lat = 52.520008
lon = 13.404954
radius = 50

query = """query {
    keywordSearch(filter: { query: "%s", lat: %s, lon: %s, radius: %s, source: EVENTS }) {
        count
        pageInfo {
            endCursor
        }
        edges {
            node {
                id
            }
        }
    }   
}""" % (activity, lat, lon, radius)
res = requests.post(base_api_url, json={'query': query}, headers=oauth_headers)

In [110]:
for events in res.json()['data']['keywordSearch']['edges']:
    event_id = events['node']['id']
    query_event = """
        query { 
            event(id: %s) { 
                title 
                description 
                dateTime 
                eventUrl
            } 
        } 
        """ % (event_id)
    res_event = requests.post(base_api_url, json={'query': query_event}, headers=oauth_headers)
    print(res_event.json()['data']['event'])

{'title': 'Berlin Outdoor Fitness Volkspark Wilmersdorf', 'description': 'Diese Gruppe ist für alle gedacht, die gerne draußen Sport treiben', 'dateTime': '2022-05-28T11:00+02:00', 'eventUrl': 'https://www.meetup.com/Berlin-Outdoor-Fitness-Volkspark-Wilmersdorf/events/285775643'}
{'title': 'Berlin Outdoor Fitness Volkspark Wilmersdorf', 'description': 'Diese Gruppe ist für alle gedacht, die gerne draußen Sport treiben', 'dateTime': '2022-05-18T18:30+02:00', 'eventUrl': 'https://www.meetup.com/Berlin-Outdoor-Fitness-Volkspark-Wilmersdorf/events/285570491'}
{'title': 'Berlin Outdoor Fitness Volkspark Wilmersdorf', 'description': 'Diese Gruppe ist für alle gedacht, die gerne draußen Sport treiben', 'dateTime': '2022-05-21T11:00+02:00', 'eventUrl': 'https://www.meetup.com/Berlin-Outdoor-Fitness-Volkspark-Wilmersdorf/events/285626483'}
{'title': 'Berlin Outdoor Fitness Volkspark Wilmersdorf', 'description': 'Diese Gruppe ist für alle gedacht, die gerne draußen Sport treiben', 'dateTime': '2

In [73]:
query_id = '''
query {
    event(id: 285856492) {
      title
      description
      dateTime
    }
  }
'''

In [74]:
res_id = requests.post(base_api_url, json={'query': query_id}, headers=oauth_headers)

In [75]:
res_id.json()

{'data': {'event': {'title': 'The First Multipotentialite Meetup Berlin',
   'description': '\\*\\* English Below \\*\\*\nBei unserem ersten Treffen wollen wir über unsere Schwierigkeiten als Vielbegabte sprechen und uns gegenseitig inspirieren. Erzähl uns, woran du gerade arbeitest, was du in deiner Freizeit machst und welchen Hindernissen du begegnest. Du darfst gern für deine Projekte werben, Kontakte knüpfen und auf neue Gedanken kommen.\n\nDas Meetup wird im Tiergarten stattfinden. Der genaue Treffpunkt wird rechtzeitig bekannt gegeben. Bringt eine Decke mit, Drinks und Snacks (wenn du magst) und dann machen wir es uns gemütlich.\n\n\\*\\*English\\*\\*\nAt our first meetup we want to discuss our difficulties as multitalented persons and inspire each other. Tell us what you are working on at the moment, what you do in your leisure time and what obstacles you encounter. You are welcome to present your projects, make contacts and come up with new ideas.\n\nThe Meetup will take place 